In [2]:
from functions import *

In [ ]:
#pip install sentinelhub

In [3]:
import matplotlib.pyplot as plt
import glob
from skimage.transform import resize
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
import geopandas as gpd
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
import matplotlib
import seaborn as sns
from sklearn.model_selection import KFold
from sklearn.preprocessing import StandardScaler
import glob
import time
from datetime import datetime
from collections import Counter
from PIL import Image
import numpy as np
from tensorflow.keras.models import load_model
from rasterio.windows import Window
from collections import Counter
from contextlib import redirect_stdout
import os
import tensorflow as tf
import geopandas as gpd

# Functions

In [ ]:
def analyze_gradients(model, test_inp_labels, test_additional_inp, river_masks):
    """
    Analyzes model gradients with respect to images and calculates the impact of variables 
    both generally and specifically for rivers.

    Args:
        model: TensorFlow model used for predictions.
        test_inp_labels: Dictionary with test images grouped by cell.
        test_additional_inp: Dictionary with additional inputs grouped by cell.
        river_masks: Dictionary with binary river masks for each cell.

    Returns:
        grad_maps: Gradient maps for each variable and cell.
        variable_impact_general: General impact (magnitude and direction) per variable and cell.
        variable_impact_river: River-specific impact (magnitude and direction) per variable and cell.
        cell_grad_analysis: Summary of general impact per cell and variable.
        river_grad_analysis: Summary of river-specific impact per cell and variable.
    """
    # Initialize data structures
    grad_maps = {var: {} for var in ["LST", "NDVI", "Slope", "Direction", "Altitude"]}
    variable_impact_general = {var: {} for var in grad_maps}
    variable_impact_river = {var: {} for var in grad_maps}
    cell_grad_analysis = {}
    river_grad_analysis = {}

    # Iterate through cells and test images
    for cell, test_images in test_inp_labels.items():
        cell_grad_analysis[cell] = {}
        river_grad_analysis[cell] = {}

        # Expand binary river mask
        river_mask = np.pad(river_masks[cell], pad_width=1, mode='constant')
        expanded_mask = (river_mask[:-2, :-2] | river_mask[1:-1, :-2] | river_mask[:-2, 1:-1] | 
                         river_mask[1:-1, 1:-1] | river_mask[:-2, 2:] | river_mask[1:-1, 2:])

        for i, test_image in enumerate(test_images):
            # Convert the image and additional inputs to tensors
            test_image_tensor = tf.expand_dims(tf.convert_to_tensor(test_image, dtype=tf.float32), axis=0)
            add_input = tf.expand_dims(tf.convert_to_tensor(test_additional_inp[cell][i], dtype=tf.float32), axis=0)

            # Compute gradients using GradientTape
            with tf.GradientTape() as tape:
                tape.watch(test_image_tensor)
                preds = model([test_image_tensor, add_input], training=False)
            gradients = tape.gradient(preds[:, 0], test_image_tensor).numpy()[0]

            # Process gradients for each variable
            for variable, channels in {"LST": range(3), "NDVI": [3], "Slope": [4], "Direction": [5], "Altitude": [6]}.items():
                grads_map = gradients[:, :, channels].mean(axis=-1)  # Combine channels
                grad_maps[variable].setdefault(cell, []).append(grads_map)

        # Calculate impacts for each variable
        for variable in grad_maps:
            grad_images = np.array(grad_maps[variable][cell])
            mean_grad = grad_images.mean(axis=0)

            # General impacts
            general_impact = np.abs(mean_grad).mean()
            general_direction = np.sign(mean_grad).mean()

            # River-specific impacts
            river_impact = np.abs(mean_grad[expanded_mask]).mean()
            river_direction = np.sign(mean_grad[expanded_mask]).mean()

            # Store results
            variable_impact_general[variable][cell] = (general_impact, general_direction)
            variable_impact_river[variable][cell] = (river_impact, river_direction)

            influence = "increases" if general_direction > 0 else "decreases"
            cell_grad_analysis[cell][variable] = {'magnitude': general_impact, 'direction': general_direction}
            river_grad_analysis[cell][variable] = {'magnitude': river_impact, 'direction': river_direction}

    return grad_maps, variable_impact_general, variable_impact_river, cell_grad_analysis, river_grad_analysis



def map_season(date):
    """Maps a date to a season based on the month."""
    month = pd.to_datetime(date).month
    if month in [12, 1, 2]:
        return 'Winter'
    elif month in [3, 4, 5, 9, 10, 11]:
        return 'Spring-Autumn'
    elif month in [6, 7, 8]:
        return 'Summer'

def analyze_gradients_by_season(grad_folder, test_inp_labels, river_masks_cut, W=64):
    """
    Analyzes gradients by cell and season, calculating mean magnitude and direction.
    """
    seasonal_grad_analysis = {'Winter': {}, 'Spring-Autumn': {}, 'Summer': {}}

    for cell, images_info in test_inp_labels.items():
        cell_grad_folder = os.path.join(grad_folder, cell)
        grad_images_by_season = {'Winter': [], 'Spring-Autumn': [], 'Summer': []}

        for grad_file_name in os.listdir(cell_grad_folder):
            grad_file_path = os.path.join(cell_grad_folder, grad_file_name)

            if grad_file_name.endswith('.npy'):
                try:
                    grad_array = np.load(grad_file_path)
                    if grad_array.shape != (W, W):  # Resize if necessary
                        grad_array = tf.image.resize(grad_array[..., np.newaxis], (W, W)).numpy().squeeze()

                    date = images_info[grad_file_name]['date']
                    season = map_season(date)
                    grad_images_by_season[season].append(grad_array)
                except Exception as e:
                    print(f"Error loading {grad_file_name}: {e}")
                    continue

        for season, grad_images in grad_images_by_season.items():
            if grad_images:
                grad_images = np.array(grad_images)
                mean_grad = np.mean(grad_images, axis=0)
                mean_magnitude = np.mean(np.abs(mean_grad))
                mean_direction = np.mean(np.sign(mean_grad))

                seasonal_grad_analysis[season][cell] = {
                    'magnitude': mean_magnitude,
                    'direction': mean_direction,
                    'mean_grad': mean_grad
                }
                print(f"Cell: {cell}, Season: {season}, Mean Magnitude: {mean_magnitude:.4f}, Direction: {mean_direction}")

    return seasonal_grad_analysis

def analyze_and_visualize_influences(cell_grad_analysis, season):
    """
    Analyzes gradient influences and visualizes results.
    """
    increase_cells = {cell: data for cell, data in cell_grad_analysis.items() if data["direction"] > 0}
    decrease_cells = {cell: data for cell, data in cell_grad_analysis.items() if data["direction"] < 0}

    increase_magnitudes = [data["magnitude"] for data in increase_cells.values()]
    decrease_magnitudes = [data["magnitude"] for data in decrease_cells.values()]

    increase_mean = np.mean(increase_magnitudes)
    increase_std = np.std(increase_magnitudes)
    increase_threshold = increase_mean + increase_std

    decrease_mean = np.mean(decrease_magnitudes)
    decrease_std = np.std(decrease_magnitudes)
    decrease_threshold = decrease_mean + decrease_std

    large_increase_cells = {cell: data["magnitude"] for cell, data in increase_cells.items() if data["magnitude"] > increase_threshold}
    large_decrease_cells = {cell: data for cell, data in decrease_cells.items() if data["magnitude"] > decrease_threshold}

    # Visualize results
    plot_influences(increase_cells, increase_magnitudes, increase_mean, increase_threshold, "Increase", season)
    plot_influences(decrease_cells, decrease_magnitudes, decrease_mean, decrease_threshold, "Decrease", season)

    return large_decrease_cells, decrease_magnitudes

def plot_influences(cells, magnitudes, mean, threshold, label, season):
    """
    Plots gradient magnitudes for the specified label (Increase/Decrease) and season.
    """
    plt.figure(figsize=(8, 5))
    plt.bar(cells.keys(), magnitudes, label=label)
    plt.axhline(y=mean, color="r", linestyle="--", label="Mean Magnitude")
    plt.axhline(y=threshold, color="g", linestyle="--", label="Large Threshold")
    plt.title(f"Gradient Magnitudes for {label} Cells ({season})")
    plt.xlabel("Cells")
    plt.ylabel("Magnitude")
    plt.legend()
    plt.show()

def save_top_25_percent_means(large_decrease_cells, river_masks_cut, season, output_folder="output"):
    """
    Saves the mean gradients of the top 25% negative influence cells as images and stores metadata.

    Args:
        large_decrease_cells (dict): Cells with the largest negative influence.
        river_masks_cut (dict): River masks for each cell.
        season (str): Current season being processed.
        output_folder (str): Folder to save the raster outputs.
    """
    if not os.path.exists(output_folder):
        os.makedirs(output_folder)

    top_25_metadata = []
    total_cells = len(large_decrease_cells)
    for cell, data in large_decrease_cells.items():
        mean_grad = data['mean_grad']
        river_mask = river_masks_cut[cell]
        river_mean_grad = mean_grad * river_mask  # Apply river mask

        # Save the raster image
        output_path = os.path.join(output_folder, f"{cell}_{season}_mean_grad.tif")
        with rasterio.open(
            output_path,
            "w",
            driver="GTiff",
            height=river_mean_grad.shape[0],
            width=river_mean_grad.shape[1],
            count=1,
            dtype=river_mean_grad.dtype
        ) as dst:
            dst.write(river_mean_grad, 1)

        # Save metadata
        top_25_metadata.append({'cell': cell, 'season': season, 'path': output_path})

    # Print summary
    print(f"Season: {season}, Total cells in top 25%: {len(large_decrease_cells)}/{total_cells}")
    return top_25_metadata
def root_mean_squared_error(y_true, y_pred):
    """Calculate RMSE."""
    return sqrt(mean_squared_error(y_true, y_pred))

def map_season(date):
    """Maps a date to a season based on the month."""
    month = pd.to_datetime(date).month
    if month in [12, 1, 2]:
        return 'Winter'
    elif month in [3, 4, 5, 9, 10, 11]:
        return 'Spring-Autumn'
    elif month in [6, 7, 8]:
        return 'Summer'

def save_gradient_images(output_folder, grad_maps, season):
    """Saves gradient maps as images for visualization."""
    os.makedirs(output_folder, exist_ok=True)
    for variable, cells in grad_maps.items():
        for cell, gradients in cells.items():
            mean_grad = np.mean(gradients, axis=0)
            output_path = os.path.join(output_folder, f"{season}_{variable}_{cell}.png")
            plt.imshow(mean_grad, cmap="bwr")
            plt.title(f"Gradient Map: {variable} ({cell}) - {season}")
            plt.colorbar()
            plt.savefig(output_path)
            plt.close()

def save_top_25_percent_means(large_decrease_cells, river_masks_cut, season, output_folder="results/top_25"):
    """
    Saves mean gradients for top 25% negative influence cells as images and stores metadata.
    """
    os.makedirs(output_folder, exist_ok=True)
    top_25_metadata = []
    total_cells = len(large_decrease_cells)

    for cell, data in large_decrease_cells.items():
        mean_grad = data['mean_grad']
        river_mask = river_masks_cut[cell]
        river_mean_grad = mean_grad * river_mask  # Apply river mask

        # Save the raster image
        output_path = os.path.join(output_folder, f"{cell}_{season}_mean_grad.tif")
        with rasterio.open(
            output_path,
            "w",
            driver="GTiff",
            height=river_mean_grad.shape[0],
            width=river_mean_grad.shape[1],
            count=1,
            dtype=river_mean_grad.dtype
        ) as dst:
            dst.write(river_mean_grad, 1)

        # Save metadata
        top_25_metadata.append({'cell': cell, 'season': season, 'path': output_path})

    # Print summary
    print(f"Season: {season}, Total cells in top 25%: {len(large_decrease_cells)}/{total_cells}")
    return top_25_metadata


# Load model

In [4]:
tf.get_logger().setLevel('ERROR') 
model_name = 'simple_CNN'
model = load_model(f'../models/{model_name}.h5')

In [8]:
summary_file = f"../models/{model_name}_summary.txt"
with open(summary_file, "w") as f:
    with redirect_stdout(f):
        model.summary()

Model: "functional_4"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ Image_Input (InputLayer)        │ (None, 128, 128, 1)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_12 (Conv2D)              │ (None, 126, 126, 16)   │           160 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_12 (MaxPooling2D) │ (None, 63, 63, 16)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_13 (Conv2D)              │ (None, 61, 61, 32)     │         4,640 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_13 (MaxPooling2D) │ (None, 30, 30, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_14 (Conv2D)              │ (None, 28, 28, 64)     │        18,496 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_14 (MaxPooling2D) │ (None, 14, 14, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten_4 (Flatten)             │ (None, 12544)          │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_8 (Dense)                 │ (None, 64)             │       802,880 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_9 (Dense)                 │ (None, 1)              │            65 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 826,241 (3.15 MB)

 Trainable params: 826,241 (3.15 MB)

 Non-trainable params: 0 (0.00 B)

# Load and prepare data

In [ ]:
datasets = {}
inputs = ['lst', 'ndvi','slope','altitude','direction']
W = 128
var_channels = {'lst': 0, 'ndvi': 1, 'slope': 2, 'altitude': 3, 'direction': 4}
var_position = {'month': [0, 1], 'coords': [2, 3], 'discharge': 4}

for split in range(1, 6):
    data_folder = f'../data/processed_data/{W}x{W}/{split}'
    test_model_input, test_additional_inputs, test_target = load_set(data_folder, inputs, 'test', var_channels, var_position)
    datasets[f'split_{split}'] = [test_model_input, test_additional_inputs, test_target]

## Load river files

In [ ]:
river_cells = {}
output_directory = '../data/external/shp/river_cells_oficial'
a=0

for i,riv in enumerate(os.listdir(output_directory)):
    # Check if the file is a shapefile
    try:
        if riv.endswith('.shp'):
            a+=1
            print(f"Processing file: {riv}")
            river = gpd.read_file(os.path.join(output_directory, riv))
            river = river.to_crs("EPSG:4326")
            river_cells[riv.split('station_')[-1].split('.')[0]] = river
            print('Added', riv.split('station_')[-1].split('.')[0])

    except Exception as e:
        print(f"Error processing file: {riv}",e)
        pass

## Load raster files

In [ ]:
import os

dir_path = '../data/external/raster_masks/'
files = os.listdir(dir_path)
river_masks = {}

for raster in files:
    cc = raster.split('bw_')[-1].split('.')[0]
    raster_path = os.path.join(dir_path, raster)
    rast, _ = load_raster(raster_path, False)
    river_masks[cc] = rast

# Model prediction

### Load labels

In [ ]:
disch_file = '../data/raw/disch/discharge.csv'
disch_df = pd.read_csv(disch_file)

labels_ordered = {}
for k, v in datasets.items():
    labels = []
    additional_inputs = v[1]
    disch_vector = additional_inputs[4]  # var_position = {'month': [0, 1], 'coords': [2, 3], 'discharge': 4}

    for disch in disch_vector:
        # Buscar en todas las columnas del DataFrame si el valor `disch` está presente
        for column in disch_df.columns:
            if disch in disch_df[column].values:
                labels.append(column)
    
    labels_ordered[k] = labels


### Results obtention pipeline

In [ ]:
# Main pipeline
results_folder = "results"
os.makedirs(results_folder, exist_ok=True)
metrics_by_splits = {}

for k, v in datasets.items():
    print('Doing split ', k)
    test_input = v[0]
    additional_inputs_test = v[1]
    test_target = v[2]

    y_pred = model.predict(test_input)

    y_test = np.array(test_target).flatten()
    y_pred = np.array(y_pred).flatten()

    # Compute metrics
    mae = mean_absolute_error(y_test, y_pred)
    mse = mean_squared_error(y_test, y_pred)
    rmse = root_mean_squared_error(y_test, y_pred)

    print(f"Mean Absolute Error (MAE): {mae}")
    print(f"Mean Squared Error (MSE): {mse}")
    print(f"Root Mean Squared Error (RMSE): {rmse}")
    metrics_by_splits[k] = {'MAE': mae, 'MSE': mse, 'RMSE': rmse}

    # Prepare data for gradient analysis
    test_inp_labels = {}
    test_dates = {}
    test_additional_inp = {}

    for i, l in enumerate(labels_ordered[k]):
        if l not in test_inp_labels:
            test_inp_labels[l] = [test_input[i]]
            test_additional_inp[l] = [additional_inputs_test[i]]
        else:
            test_inp_labels[l].append(test_input[i])
            test_additional_inp[l].append(additional_inputs_test[i])

    # Analyze gradients
    grad_maps, variable_impact_general, variable_impact_river, cell_grad_analysis, river_grad_analysis = analyze_gradients(
        model=model,
        test_inp_labels=test_inp_labels,
        test_additional_inp=test_additional_inp,
        river_masks_cut=river_masks
    )

    # Analyze by season
    seasonal_grad_analysis = analyze_gradients_by_season(
        grad_folder="../plots/grad_cam/simple_CNN",
        test_inp_labels=test_inp_labels,
        river_masks_cut=river_masks,
        W=64
    )

    # Save gradient images
    save_gradient_images(output_folder=os.path.join(results_folder, f"gradients_{k}"), grad_maps=grad_maps, season=k)

    # Analyze and visualize influences
    for season, season_data in seasonal_grad_analysis.items():
        increase_cells, decrease_cells = analyze_and_visualize_influences(cell_grad_analysis=season_data, season=season)

        # Filter and save top 25% of cells with negative influence
        top_25_metadata = save_top_25_percent_means(
            large_decrease_cells=decrease_cells,
            river_masks_cut=river_masks,
            season=season,
            output_folder=os.path.join(results_folder, "top_25")
        )

        # Save metadata
        with open(os.path.join(results_folder, f"{season}_top_25_metadata.txt"), "w") as f:
            for item in top_25_metadata:
                f.write(f"Cell: {item['cell']}, Season: {item['season']}, Path: {item['path']}\n")


# Download true color photos

In [ ]:
slots = []

for date in unique_dates:
  year = int(date.split('-')[0])
  month = int(date.split('-')[1])
  fd = first_day(month, year)
  ld = last_day(fd)
  slots.append( (fd.isoformat(), ld.isoformat()) )

slots

In [ ]:
evalscript = """
    //VERSION=3

    let minVal = 0.0;
    let maxVal = 0.4;

    let viz = new DefaultVisualizer(minVal, maxVal);

    function evaluatePixel(samples) {
        let val = [samples.B04, samples.B03, samples.B02];
        val = viz.processList(val);
        val.push(samples.dataMask);
        return val;
    }

    function setup() {
    return {
        input: [{
        bands: [
            "B02",
            "B03",
            "B04",
            "dataMask"
        ]
        }],
        output: {
        bands: 4
        }
    }
    }"""

In [ ]:
slots = [('2013-06-01', '2013-06-30')]
river = rivers['cell_163']
get_data(river, evalscript, slots, 'lst', destination_folder)
    